In [ ]:
import datetime as dt 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import MetaTrader5 as mt5
import matplotlib.dates as mdates
import yfinance as yf
import os
import talib


In [ ]:
# Definir a data para coleta de dados

data_inicial = dt.datetime(2023, 1, 2)   
data_final = dt.datetime(2023, 11, 29)   #META TRADER TEM LIMITE DE DIAS 

# Definir a data para coleta de dados

data_inicial2 = dt.datetime(2023, 11, 30)   
data_final2 = dt.datetime(2024, 10, 31)   #META TRADER TEM LIMETE DE DIAS 



In [ ]:
# Função para calcular médias móveis
def moving_averages(data, Fast, Slow):
    
    data['sma_slow'] = data['Price'].rolling(Slow).mean()
    data['sma_fast'] = data['Price'].ewm(span=Fast).mean()
    data['RSI'] = talib.RSI(data['Price'], timeperiod=14)
    return data

# Função para identificar cruzamentos
def find_crossover(fast_sma, slow_sma):
    if fast_sma > slow_sma:
        return 'BUY'
    elif fast_sma < slow_sma:
        return 'SELL'
    else:
        return 'HOLD'

symbol = "WIN$"
taxa = 0.2
time_frame = mt5.TIMEFRAME_M5
Fast = 9
Slow = 20
stop_loss = 400
stop_gain = 1200

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print("O ativo especificado não foi encontrado")
    mt5.shutdown()
    quit()

if not symbol_info.visible:
    print(symbol, "não está visível, tentando alterar...")
    if not mt5.symbol_select(symbol, True):
        print("Falha ao selecionar", symbol)
        mt5.shutdown()
        quit()

Ticks1 = mt5.copy_rates_range(symbol, time_frame, data_inicial, data_final)
data1 = pd.DataFrame(columns=['Time','Price'])
data1['Price'] = Ticks1['close']
data1['Time'] = Ticks1['time']
data1['Time'] = pd.to_datetime(data1['Time'], unit='s')
data1['Price'] = pd.to_numeric(data1['Price'])
data1['High'] = Ticks1['high']
data1['Low'] = Ticks1['low']
# data = data1

Ticks2 = mt5.copy_rates_range(symbol, time_frame, data_inicial2, data_final2)
data2 = pd.DataFrame(columns=['Time','Price'])
data2['Price'] = Ticks2['close']
data2['Time'] = Ticks2['time']
data2['Time'] = pd.to_datetime(data2['Time'], unit='s')
data2['Price'] = pd.to_numeric(data2['Price'])
data2['High'] = Ticks2['high']
data2['Low'] = Ticks2['low']

data = pd.concat([data1, data2], axis=0)
data = data.reset_index(drop=True)



In [ ]:

def GO_order(Performance, symbol, buy_time, entry_price, Status, lado):

    Operation = len(Performance) + 1
    lot = 1
    Performance = Performance._append({'Status': Status,
                                    'Numero': Operation,
                                    'Ativo': symbol,
                                    'Dia': buy_time,
                                    'Hora Abertura': buy_time,
                                    'Qtde': lot,
                                    'Lado': lado,
                                    'Preco de entrada': entry_price},
                                    ignore_index=True)

    Performance['Dia'] = pd.to_datetime(Performance['Dia']).dt.date
    return Performance

def OUT_order(Performance, exit_price, sell_time, Status, taxa):
    
    Performance.loc[Performance['Status'] == 'ABERTA', 'Hora Fechamento'] = sell_time
    Performance.loc[Performance['Status'] == 'ABERTA', 'Preco de saida'] = exit_price
    
    # Converta as colunas para float se necessário
    Performance['Preco de entrada'] = Performance['Preco de entrada'].astype(float)
    Performance['Preco de saida'] = Performance['Preco de saida'].astype(float)

    # Atualizar 'Resultado Intervalo' de forma condicional para todas as linhas com 'Status' igual a 'ABERTA'
    Resultado = np.where(
        Performance.loc[Performance['Status'] == 'ABERTA', 'Lado'] == 'Compra',
        Performance.loc[Performance['Status'] == 'ABERTA', 'Preco de saida'] - Performance.loc[Performance['Status'] == 'ABERTA', 'Preco de entrada'],
        Performance.loc[Performance['Status'] == 'ABERTA', 'Preco de entrada'] - Performance.loc[Performance['Status'] == 'ABERTA', 'Preco de saida']
    )

    # Aplicar o resultado na coluna 'Resultado Intervalo'    
    Performance.loc[Performance['Status'] == 'ABERTA', 'Resultado Intervalo'] = Resultado
    Performance.loc[Performance['Status'] == 'ABERTA', 'Resultado %'] = ((exit_price / Performance.loc[Performance['Status'] == 'ABERTA', 'Preco de entrada']) - 1) * 100
    Performance.loc[Performance['Status'] == 'ABERTA', 'Resultado Financeiro'] = Performance.loc[Performance['Status'] == 'ABERTA', 'Resultado Intervalo'] * taxa
    Performance.loc[Performance['Status'] == 'ABERTA', 'Resultado Acumulado'] = Performance.loc[Performance['Status'] == 'ABERTA', 'Resultado Acumulado'].shift(1).fillna(0) + Performance.loc[Performance['Status'] == 'ABERTA', 'Resultado Financeiro']

    try:
        Performance['Hora Fechamento'] = pd.to_datetime(Performance['Hora Fechamento'])
        Performance['Hora Abertura'] = pd.to_datetime(Performance['Hora Abertura'])
    except Exception as e:
        Performance['Hora Fechamento'].iloc[-1] = "Erro ao converter horários"
        
        
    Performance.loc[Performance['Status'] == 'ABERTA', 'Tempo de Operacao'] = Performance['Hora Fechamento'] - Performance['Hora Abertura']
    Performance.loc[Performance['Status'] == 'ABERTA', 'Status'] = Status
    
    return Performance

In [ ]:
# Plotar dados

plt.figure(figsize=(20, 8))

plt.plot(data['Time'], data['Price'], label='Price', color='k')

# Converter o índice em uma lista de datas para os rótulos do eixo x
x_labels = data.index.to_list()

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.xlabel('Time',)
plt.ylabel('Price')
plt.title('Preço do mini índice - 2023',fontsize=20)
plt.legend()


# Reduzir a quantidade de rótulos do eixo x (por exemplo, mostrando apenas a cada hora)
#plt.xticks(data.index[::3600], labels=data['Time'].dt.strftime('%d | %H:%M')[::3600], rotation=45)

fig_path = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/PricaOverTime_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path)
plt.grid(True)
plt.show()


In [ ]:

# Calcular médias móveis
data = moving_averages(data, Fast, Slow)


data['Recomendação'] = np.vectorize(find_crossover)(data['sma_fast'], data['sma_slow'])

data['Signal'] = 0

data['Signal'] = np.where(data['sma_fast'] > data['sma_slow'], 1, 0)

data['Position'] = data['Signal'].diff()

data['Status'] = 0

data['OBS'] = '0'

# Inicializar dataframe de performance
Performance = pd.DataFrame(columns=['Numero',   
                                    'Status', 
                                    'Ativo', 
                                    'Dia', 
                                    'Hora Abertura',
                                    'Hora Fechamento',
                                    'Tempo de Operacao', 
                                    'Qtde', 
                                    'Lado', 
                                    'Preco de entrada', 
                                    'Preco de saida', 
                                    'Resultado Intervalo', 
                                    'Resultado %', 
                                    'Resultado Financeiro', 
                                    'Resultado Acumulado'])


In [ ]:

# Ajustar o tamanho da figura
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 10), sharex=True)

# Primeiro gráfico: Preço e Médias Móveis
ax1.plot(data['Time'], data['Price'], label='Price', color='k')
ax1.plot(data['Time'], data['sma_fast'], label='sma_fast', color='blue')  # Exemplo de média móvel
ax1.plot(data['Time'], data['sma_slow'], label='sma_slow', color='red')   # Outra média móvel
ax1.set_title('Preço do mini índice - 2023', fontsize=20)
ax1.set_ylabel('Price')
ax1.legend()
ax1.grid(True)

# Segundo gráfico: RSI
ax2.plot(data['Time'], data['RSI'], label='RSI', color='purple')
ax2.axhline(70, color='r', linestyle='--', alpha=0.5)  # Nível de sobrecompra
ax2.axhline(30, color='g', linestyle='--', alpha=0.5)  # Nível de sobrevenda
ax2.set_ylabel('RSI')
ax2.set_xlabel('Time')
ax2.legend()
ax2.grid(True)

# Ajustar rótulos do eixo x
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
#### PREÇO E MEDIAS MOVEIS #####

# plt.figure(figsize=(20, 8))

# plt.plot(data.index, data['Price'], label='Price', color='k')

# plt.plot(data.index, data['sma_fast'], label='sma_fast', color='b')
# plt.plot(data.index, data['sma_slow'], label='sma_slow', color='g')

# # Converter o índice em uma lista de datas para os rótulos do eixo x
# x_labels = data.index.to_list()

# plt.xlabel('Time')
# plt.ylabel('Price')
# plt.title('Price over Time')
# plt.legend()

# # Reduzir a quantidade de rótulos do eixo x (por exemplo, mostrando apenas a cada hora)
# #plt.xticks(data.index[::3600], labels=data['Time'].dt.strftime('%d | %H:%M')[::3600], rotation=45)

# plt.grid(True)
# plt.show()

In [ ]:
# Inicializar variáveis
status_operation = 'FECHADA'
order_type = 0
entry_price = 0
operation_number = 0

# Iterar sobre os dados
for i in range(len(data)):

    # Verificar condição de venda (stop loss, stop gain, ou fechamento de mercado)

    if status_operation == 'ABERTA':
        
        resultado = (data['Price'].iloc[i] - entry_price) if order_type == 'Compra' else (entry_price - data['Price'].iloc[i])

        print(f'Operação de {order_type} e Resultado(pts): {resultado}')
        
        if resultado <= -stop_loss:
            
            status_operation = 'FECHADA'
            sell_price = data['Price'].iloc[i]
            sell_time = data['Time'].iloc[i]
            data.loc[i, 'Status'] = -1 if order_type == 'Compra' else 1
            data.at[i,'OBS'] = f'Stop Loss acionado na linha {i} - valor: {stop_loss}'
            Performance = OUT_order(Performance, sell_price, sell_time, status_operation, taxa)

        elif resultado >= stop_gain:
            
            status_operation = 'FECHADA'
            sell_price = data['Price'].iloc[i]
            sell_time = data['Time'].iloc[i]
            data.at[i,'OBS'] = f'Stop Gain acionado na linha {i}'
            data.loc[i, 'Status'] = -1 if order_type == 'Compra' else 1
            Performance = OUT_order(Performance, sell_price, sell_time, status_operation, taxa)

        elif data['Time'].iloc[i].hour == 17 and data['Time'].iloc[i].minute > 45:

            data.at[i,'OBS'] = f'Fechamento de mercado na linha {i}'
            status_operation = 'FECHADA'
            
            data.loc[i, 'Status'] = -1 if order_type == 'Compra' else 1
            sell_price = data['Price'].iloc[i]
            sell_time = data['Time'].iloc[i]
            Performance = OUT_order(Performance, sell_price, sell_time, status_operation, taxa)

    
    if  ((data['Time'].iloc[i].hour == 9 and data['Time'].iloc[i].minute >= 5) or
                    (data['Time'].iloc[i].hour > 9 and data['Time'].iloc[i].hour < 17) or
                    (data['Time'].iloc[i].hour == 17 and data['Time'].iloc[i].minute <= 45)):
     
        ###### OPERAÇÃO - TRADING #######
       
        if pd.notna(data['sma_fast'].iloc[i]) and pd.notna(data['sma_slow'].iloc[i]):
            
            if status_operation == 'FECHADA':
                
                # BUY ORDER - ENTRADA
                
                if data['Position'].iloc[i] == 1 and data['Signal'].iloc[i] == 1 and data['RSI'].iloc[i] < 45:

                    data.at[i,'OBS'] = f'Compra na linha {i} - BUY SIDE'
                    status_operation = 'ABERTA'
                    order_type = 'Compra'
                    entry_price = data['Price'].iloc[i]
                    buy_time = data['Time'].iloc[i]
                    data['Status'].iloc[i] = 1
                    Performance = GO_order(Performance, symbol, buy_time, entry_price, status_operation, order_type)

                # SEL ORDER - ENTRADA 
                
                elif data['Position'].iloc[i] == -1 and data['Signal'].iloc[i] == 0 and data['RSI'].iloc[i] > 60:    # Para VENDA, MELHOR RELAÇÃO É RSI > 60 
 
                    data.at[i,'OBS'] = f'Venda na linha {i} - SELL SIDE'
                    entry_price = data['Price'].iloc[i]
                    sell_time = data['Time'].iloc[i]
                    data['Status'].iloc[i] = -1
                    status_operation = 'ABERTA'
                    order_type = 'Venda'
                    Performance = GO_order(Performance, symbol, sell_time ,entry_price, status_operation, order_type)

            elif status_operation == 'ABERTA':
                
                # BUY SIDE - SAIDA
                
                if data['Position'].iloc[i] == -1 and data['Signal'].iloc[i] == 0 and order_type == 'Compra':

                    data.at[i,'OBS'] = f'Venda na linha {i} - STOP BUY SIDE'
                    status_operation = 'FECHADA'
                    buy_price = data['Price'].iloc[i]
                    buy_time = data['Time'].iloc[i]
                    data['Status'].iloc[i] = -1
                    Performance = OUT_order(Performance, buy_price, buy_time, status_operation, taxa)

                # SEL SIDE - SAIDA
                
                elif data['Position'].iloc[i] == 1 and data['Signal'].iloc[i] == 1 and order_type == 'Venda':

                    data.at[i,'OBS'] = f'Compra na linha {i} - STOP SELL SIDE'
                    sell_price = data['Price'].iloc[i]
                    sell_time = data['Time'].iloc[i]
                    status_operation = 'FECHADA'
                    data['Status'].iloc[i] = -1
                    Performance = OUT_order(Performance, sell_price, sell_time, status_operation, taxa)



In [ ]:
Performance['Resultado Acumulado'] = Performance['Resultado Financeiro'].cumsum()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Convertendo a coluna 'Time' para datetime se ainda não estiver
data['Time'] = pd.to_datetime(data['Time'])

# Filtrando os dados para o intervalo desejado
start_date = '2024-01-04'
end_date = '2024-01-05'
filtered_data = data[(data['Time'] >= start_date) & (data['Time'] < end_date)]

# Criar subplots: um para o gráfico de preço e outro para o RSI
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})


# Gráfico superior: Preço e Médias Móveis
ax1.plot(filtered_data.index, filtered_data['Price'], label='Price', color='k')
ax1.plot(filtered_data.index, filtered_data['sma_fast'], label='sma_fast', color='b')
ax1.plot(filtered_data.index, filtered_data['sma_slow'], label='sma_slow', color='g')
ax1.plot(filtered_data[filtered_data['Status'] == 1].index, filtered_data[filtered_data['Status'] == 1]['Price'], '^', markersize=15, color='g', label='Buy')
ax1.plot(filtered_data[filtered_data['Status'] == -1].index, filtered_data[filtered_data['Status'] == -1]['Price'], 'v', markersize=15, color='r', label='Sell')

# Adicionar anotações para os valores de compra e venda
for idx, row in filtered_data[filtered_data['Status'] == 1].iterrows():
    ax1.annotate(f'{row["Price"]:.2f}', (idx, row['Price']), textcoords="offset points", xytext=(0,10), ha='center', color='w',bbox=dict(facecolor='black', alpha=0.75))
for idx, row in filtered_data[filtered_data['Status'] == -1].iterrows():
    ax1.annotate(f'{row["Price"]:.2f}', (idx, row['Price']), textcoords="offset points", xytext=(0,10), ha='center', color='w',bbox=dict(facecolor='black', alpha=0.75))




# Configurações do gráfico de preço
ax1.set_ylabel('Price')
ax1.set_title('Cruzamento de médias e Operações')
ax1.legend()
ax1.grid(True)

# Formatar o eixo x para mostrar datas e horas
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
fig.autofmt_xdate()

# Gráfico inferior: RSI
ax2.plot(filtered_data.index, filtered_data['RSI'], label='RSI', color='purple')
ax2.axhline(60, color='r', linestyle='--', alpha=0.5)  # Linha de sobrecompra
ax2.axhline(45, color='g', linestyle='--', alpha=0.5)  # Linha de sobrevenda
ax2.set_ylabel('RSI')
ax2.set_xlabel('Time (d/m | H:M)')
ax2.legend()
ax2.grid(True)

# Configuração dos rótulos do eixo x
ax2.set_xticks(filtered_data.index[::5])
ax2.set_xticklabels(filtered_data['Time'].dt.strftime('%d/%m | %H:%M')[::5], rotation=45)

# Salvar o gráfico
fig_path1 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Grafico_melhor_dia_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path1)

plt.show()



In [ ]:

val_pos = Performance[Performance['Resultado Financeiro'] > 0]['Resultado Financeiro']
val_neg = Performance[Performance['Resultado Financeiro'] < 0]['Resultado Financeiro']

num_negativos = val_neg.count()
num_positivos = val_pos.count()

Media_positivos = val_pos.mean()
Media_Negativos = val_neg.mean()

Total_trades = len(Performance)
print(f'Total de Trades: {Total_trades} - Trades Positivos: {num_positivos} - Trades Negativos: {num_negativos} \nMédia $ positivos: {Media_positivos:.0f} - Média $ Negativos: {Media_Negativos:.0f}')


In [ ]:
#Curva de capital da estratégia

plt.figure(figsize=(20,8))

plt.plot(Performance['Dia'], Performance['Resultado Acumulado'], label='Resultado Acumulado', color='k')

plt.axhline(y=12, color='red')
plt.xticks(rotation=45)
plt.title('Curva do Resultado Acumulado',fontsize=20)
plt.legend()
plt.grid(True)

fig_path1 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Curva do Capital_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path1)

plt.show()

In [ ]:
# Plotar dados com dois eixos y
fig, ax1 = plt.subplots(figsize=(20, 8))

# Primeiro eixo y (para 'Resultado Acumulado')
ax1.plot(Performance['Dia'], Performance['Resultado Acumulado'], label='Resultado Acumulado', color='k')
ax1.set_xlabel('Time')
ax1.set_ylabel('Resultado Acumulado', color='k')
ax1.tick_params(axis='y', labelcolor='k')

# Segundo eixo y (para 'Price')
ax2 = ax1.twinx()
ax2.plot(data['Time'], data['Price'], color='b', label='Preço')
ax2.set_ylabel('Price', color='b')
ax2.tick_params(axis='y', labelcolor='b')

# Linha horizontal no primeiro eixo y
ax1.axhline(y=12, color='red')

# Formatar o eixo x para mostrar datas e horas
#ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d | %H:%M'))
#fig.autofmt_xdate()

# Adicionar título e legenda
plt.title(f'Curva do Capital x Preço do Ativo',fontsize=20)
fig.tight_layout()  # Ajusta o layout para evitar sobreposição

fig.legend(loc='upper left',bbox_to_anchor=(0.05,0.95), fontsize = 'x-large')

fig_path2 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Curva do Capital X Preco_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path2)

# Adicionar grade
ax1.grid(True)
plt.show()



In [ ]:
# Plotar dados com dois eixos y
fig, ax1 = plt.subplots(figsize=(20, 8))

# Primeiro eixo y (para 'Resultado Acumulado')
ax1.plot(Performance['Dia'], Performance['Resultado Acumulado'], label='Curva', color='k')
ax1.set_xlabel('Time')
ax1.set_ylabel('Resultado Acumulado', color='k')
ax1.tick_params(axis='y', labelcolor='k')

# Segundo eixo y (para 'Price')
ax2 = ax1.twinx()
ax2.plot(data['Time'], data['Price'], color='b', label='Preço')
ax2.set_ylabel('Price', color='b')
ax2.tick_params(axis='y', labelcolor='b')

# Linha horizontal no primeiro eixo y
ax1.axhline(y=0, color='red')

# Formatar o eixo x para mostrar datas e horas
#ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d | %H:%M'))
#fig.autofmt_xdate()

# Adicionar título e legenda
plt.title('Curva do Capital x Preço do Ativo',fontsize=20)
fig.tight_layout()  # Ajusta o layout para evitar sobreposição

fig.legend(loc='upper left',bbox_to_anchor=(0.05,0.95), fontsize = 'x-large')

fig_path2 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Curva do Capital X Preco_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path2)

# Adicionar grade
ax1.grid(True)
plt.show()


In [ ]:
Performance_Daily = Performance.groupby(['Dia'])['Resultado Financeiro'].agg(['sum', 'count'])

Performance_Daily['Acumulado'] = Performance_Daily['sum'].cumsum()

Performance_Daily = Performance_Daily.rename(columns={'sum':'Total'})

Performance_Daily = Performance_Daily.rename(columns={'count':'Trades'})

Performance_Daily['Investimento'] = 1000 + Performance_Daily['Acumulado']

Performance_Daily

In [ ]:
Dia_mais_neg = Performance_Daily['Total'].idxmin()

Total_neg = Performance_Daily['Total'].min()

Dia_mais_pos = Performance_Daily['Total'].idxmax()

Total_pos = Performance_Daily['Total'].max()

print(f'Pior dia: {Dia_mais_neg} Total: R$ {Total_neg:.2F}')

print(f'Melhor dia: {Dia_mais_pos} Total: R$ {Total_pos:.2F}')

In [ ]:

# Supondo que 'Dia_mais_pos' seja a variável com a data do melhor dia
end_date = Dia_mais_pos + dt.timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')
start_date = Dia_mais_pos.strftime('%Y-%m-%d')

# Convertendo a coluna 'Time' para datetime se ainda não estiver
data['Time'] = pd.to_datetime(data['Time'])

# Filtrando os dados para o intervalo desejado
filtered_data = data[(data['Time'] >= start_date) & (data['Time'] < end_date)]
# Criar subplots: um para o gráfico de preço e outro para o RSI
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})


# Gráfico superior: Preço e Médias Móveis
ax1.plot(filtered_data.index, filtered_data['Price'], label='Price', color='k')
ax1.plot(filtered_data.index, filtered_data['sma_fast'], label='sma_fast', color='b')
ax1.plot(filtered_data.index, filtered_data['sma_slow'], label='sma_slow', color='g')
ax1.plot(filtered_data[filtered_data['Status'] == 1].index, filtered_data[filtered_data['Status'] == 1]['Price'], '^', markersize=15, color='g', label='Buy')
ax1.plot(filtered_data[filtered_data['Status'] == -1].index, filtered_data[filtered_data['Status'] == -1]['Price'], 'v', markersize=15, color='r', label='Sell')

# Adicionar anotações para os valores de compra e venda
for idx, row in filtered_data[filtered_data['Status'] == 1].iterrows():
    ax1.annotate(f'{row["Price"]:.2f}', (idx, row['Price']), textcoords="offset points", xytext=(0,10), ha='center', color='w',bbox=dict(facecolor='black', alpha=0.75))
for idx, row in filtered_data[filtered_data['Status'] == -1].iterrows():
    ax1.annotate(f'{row["Price"]:.2f}', (idx, row['Price']), textcoords="offset points", xytext=(0,10), ha='center', color='w',bbox=dict(facecolor='black', alpha=0.75))




# Configurações do gráfico de preço
ax1.set_ylabel('Price')
ax1.set_title('Cruzamento de médias e Operações')
ax1.legend()
ax1.grid(True)

# Formatar o eixo x para mostrar datas e horas
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
fig.autofmt_xdate()

# Gráfico inferior: RSI
ax2.plot(filtered_data.index, filtered_data['RSI'], label='RSI', color='purple')
ax2.axhline(60, color='r', linestyle='--', alpha=0.5)  # Linha de sobrecompra
ax2.axhline(45, color='g', linestyle='--', alpha=0.5)  # Linha de sobrevenda
ax2.set_ylabel('RSI')
ax2.set_xlabel('Time (d/m | H:M)')
ax2.legend()
ax2.grid(True)

# Configuração dos rótulos do eixo x
ax2.set_xticks(filtered_data.index[::5])
ax2.set_xticklabels(filtered_data['Time'].dt.strftime('%d/%m | %H:%M')[::5], rotation=45)

# Salvar o gráfico
fig_path3 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Grafico_melhor_dia_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path3)

plt.show()



In [ ]:
end_date = Dia_mais_neg + dt.timedelta(days=1)

end_date = end_date.strftime('%Y-%m-%d')

start_date = Dia_mais_neg.strftime('%Y-%m-%d')

# Convertendo a coluna 'Time' para datetime se ainda não estiver
data['Time'] = pd.to_datetime(data['Time'])

# Filtrando os dados para o intervalo desejado
filtered_data = data[(data['Time'] >= start_date) & (data['Time'] < end_date)]
# Criar subplots: um para o gráfico de preço e outro para o RSI
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})


# Gráfico superior: Preço e Médias Móveis
ax1.plot(filtered_data.index, filtered_data['Price'], label='Price', color='k')
ax1.plot(filtered_data.index, filtered_data['sma_fast'], label='sma_fast', color='b')
ax1.plot(filtered_data.index, filtered_data['sma_slow'], label='sma_slow', color='g')
ax1.plot(filtered_data[filtered_data['Status'] == 1].index, filtered_data[filtered_data['Status'] == 1]['Price'], '^', markersize=15, color='g', label='Buy')
ax1.plot(filtered_data[filtered_data['Status'] == -1].index, filtered_data[filtered_data['Status'] == -1]['Price'], 'v', markersize=15, color='r', label='Sell')

# Adicionar anotações para os valores de compra e venda
for idx, row in filtered_data[filtered_data['Status'] == 1].iterrows():
    ax1.annotate(f'{row["Price"]:.2f}', (idx, row['Price']), textcoords="offset points", xytext=(0,10), ha='center', color='w',bbox=dict(facecolor='black', alpha=0.75))
for idx, row in filtered_data[filtered_data['Status'] == -1].iterrows():
    ax1.annotate(f'{row["Price"]:.2f}', (idx, row['Price']), textcoords="offset points", xytext=(0,10), ha='center', color='w',bbox=dict(facecolor='black', alpha=0.75))


# Configurações do gráfico de preço
ax1.set_ylabel('Price')
ax1.set_title('Cruzamento de médias e Operações')
ax1.legend()
ax1.grid(True)

# Formatar o eixo x para mostrar datas e horas
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
fig.autofmt_xdate()

# Gráfico inferior: RSI
ax2.plot(filtered_data.index, filtered_data['RSI'], label='RSI', color='purple')
ax2.axhline(60, color='r', linestyle='--', alpha=0.5)  # Linha de sobrecompra
ax2.axhline(45, color='g', linestyle='--', alpha=0.5)  # Linha de sobrevenda
ax2.set_ylabel('RSI')
ax2.set_xlabel('Time (d/m | H:M)')
ax2.legend()
ax2.grid(True)

# Configuração dos rótulos do eixo x
ax2.set_xticks(filtered_data.index[::5])
ax2.set_xticklabels(filtered_data['Time'].dt.strftime('%d/%m | %H:%M')[::5], rotation=45)


fig_path4 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Grafico_pior_dia_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path4)

plt.show()

In [ ]:


# Definir o símbolo do índice Ibovespa e o período de interesse
symbol = '^BVSP'
start_date = '2023-01-01'
end_date = '2024-10-31'

# Obter os dados históricos do Yahoo Finance
Ibov = yf.download(symbol, start=start_date, end=end_date)['Adj Close']

Ibov = pd.DataFrame(Ibov)

Ibov = Ibov.rename({'Date':'Time'})

Ibov['Daily Return'] = Ibov['Adj Close'].pct_change()
Ibov['Daily Cumulative'] = (1+Ibov['Daily Return']).cumprod()

Ibov['Investimento'] = 1000 * Ibov['Daily Cumulative']
Ibov


In [ ]:
plt.figure(figsize=(20,8))
plt.plot(Ibov['Adj Close'], color='k')

plt.xticks(rotation=45)
plt.title('Ibovespa',fontsize=20)
plt.grid(True)

fig_path5 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Grafico_ibovespa_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path5)

plt.show()

In [ ]:
Investimentos = pd.merge(Performance_Daily['Investimento'] ,Ibov['Investimento'], left_index=True, right_index=True, how='outer',suffixes=('Performance', 'Ibovespa'))

Investimentos.fillna(method='ffill', inplace=True)

plt.figure(figsize=(20,8))
plt.plot(Investimentos, color='k')

plt.xticks(rotation=45)
plt.title('Ibovespa vs Algo Trading',fontsize=20)
plt.grid(True)

fig_path6 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Ibovespa vs Algo Trading_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path6)

plt.show()

In [ ]:
Investimentos.tail(10)

In [ ]:
num_negativos = (Performance_Daily['Total'] < 0).sum()

num_positivos = (Performance_Daily['Total'] > 0).sum()

Total_trades = len(Performance_Daily)

print(f'Total Dias: {Total_trades} - Dias Positivos:{num_positivos} - Dias Negativos: {num_negativos}')

plt.figure(figsize=(20,8))
plt.bar(Performance_Daily.index, Performance_Daily['Total'])

fig_path7 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Perfomance_Daily_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path7)

plt.show()

In [ ]:
plt.figure(figsize=(20,8))
plt.hist(Performance_Daily['Total'], bins=15)
plt.grid(True)
plt.show()

In [ ]:

plt.figure(figsize=(20,8))

plt.plot(Performance_Daily.index, Performance_Daily['Acumulado'], color='k')

plt.bar(Performance_Daily.index, Performance_Daily['Total'])

plt.title('Resultado por dia e Curva de Capital')

fig_path8 = f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Resultado Dia e Curva_{dt.datetime.now().strftime("%d%m-%H%M%S")}.png'
plt.savefig(fig_path8)

plt.show()

In [ ]:
# Save Data and Performance to Excel
with pd.ExcelWriter(f'C:/Users/paulo/Desktop/UFSM - 2024/Projeto/Outputs/Backtest_{dt.datetime.now().strftime("%d%m-%H%M%S")}.xlsx', engine='xlsxwriter') as writer:

    data.to_excel(writer, sheet_name='Data', index=False)

    Performance.to_excel(writer, sheet_name='Performance', index=False)

    Performance_Daily.to_excel(writer, sheet_name='Performance_Daily_table')

    Investimentos.to_excel(writer, sheet_name='Investimento_table')

    Ibov.to_excel(writer, sheet_name='Ibovespa_table')
    
    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Preço')
    writer.sheets['Preço'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path)

    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Curva do Capital')
    writer.sheets['Curva do Capital'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path1)

    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Preço x Curva')
    writer.sheets['Preço x Curva'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path2)

    
    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Gráfico do Melhor dia')
    writer.sheets['Gráfico do Melhor dia'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path3)

    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Gráfico do Pior dia')
    writer.sheets['Gráfico do Pior dia'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path4)

    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Ibovespa')
    writer.sheets['Ibovespa'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path5)
    
    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Ibovespa vs Algo Trading')
    writer.sheets['GIbovespa vs Algo Trading'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path6)

    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Perfomance_Daily')
    writer.sheets['Perfomance_Daily'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path7)

    # Save the figure
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Resultado e Curva')
    writer.sheets['Resultado e Curva'] = worksheet

    # Insert image into Excel sheet
    worksheet.insert_image('B2', fig_path8)


In [ ]:
os.remove(fig_path)
os.remove(fig_path1)
os.remove(fig_path2)
os.remove(fig_path3)
os.remove(fig_path4)
os.remove(fig_path5)
os.remove(fig_path6)
os.remove(fig_path7)
os.remove(fig_path8)